In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("Train.csv")
df.head(2)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228


In [3]:
# total missing values in Item_Weight column before imputation

df.Item_Weight.isnull().sum()

1463

In [4]:
# Example reference

(df.loc[df['Item_Identifier'] == "FDX07"]).iloc[:, [0,1]]

,Item_Identifier,Item_Weight
3,FDX07,19.2
1491,FDX07,19.2
2459,FDX07,19.2
3089,FDX07,19.2
5906,FDX07,19.2
7735,FDX07,NaN


In [5]:
(df.loc[df['Item_Identifier'] == "FDE51"]).iloc[:, [0,1]]

,Item_Identifier,Item_Weight
28,FDE51,5.925
755,FDE51,5.925
2411,FDE51,5.925
5427,FDE51,NaN
5499,FDE51,5.925
6531,FDE51,5.925
7315,FDE51,NaN


### How we are going to fill missing values ?
    From analysis we identified that the weights are exactly same for unique items. 
    Say for example, Item_Identifier - 'FDX07' its corresponding 'Item_Weight' is 19.2

In [6]:
# sort data based of item_weight
temp1 = df.sort_values(by=['Item_Weight'])

# get unique Item_Identifiers and its corresponding Item_Weight
uniqueItemIdentifiers = (temp1.drop_duplicates(subset ="Item_Identifier")).iloc[:, [0,1]]

# covert it to dictinary with identifier as key and weight as value.
uniqueIdentifiers = uniqueItemIdentifiers.set_index('Item_Identifier')['Item_Weight'].to_dict()

# fill missing weights
for i in range(0, len(df)):
    if np.isnan(df.at[i, 'Item_Weight']):
        df.at[i, 'Item_Weight'] = uniqueIdentifiers[str(df.at[i, 'Item_Identifier'])]

In [7]:
# total missing values in Item_Weight column after imputation

df.Item_Weight.isnull().sum()

4

From the above output cell,
### Information 1
    we can infer that there are 4 missing values in Item_Weight column after imputation.
    i.e. 4 item_identifier's doesn't have item_weight in the given data.

In [8]:
# Item_Identifier's that doesn't have Item_Weight

(df[df['Item_Weight'].isna()])['Item_Identifier'].unique()

array(['FDN52', 'FDK57', 'FDE52', 'FDQ60'], dtype=object)

In [9]:
# (df.loc[df['Item_Identifier'].isin(["FDN52", "FDK57", "FDE52", "FDQ60"])]).iloc[:, [0,1]]
(df.loc[df['Item_Identifier'].isin(["FDN52", "FDK57", "FDE52", "FDQ60"])])

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
927,FDN52,NaN,Regular,0.130933,Frozen Foods,86.9198,OUT027,1985,Medium,Tier 3,Supermarket Type3,1569.9564
1922,FDK57,NaN,Low Fat,0.079904,Snack Foods,120.0440,OUT027,1985,Medium,Tier 3,Supermarket Type3,4434.2280
4187,FDE52,NaN,Regular,0.029742,Dairy,88.9514,OUT027,1985,Medium,Tier 3,Supermarket Type3,3453.5046
5022,FDQ60,NaN,Regular,0.191501,Baking Goods,121.2098,OUT019,1985,Small,Tier 1,Grocery Store,120.5098


From the above cell, Item identifier's which doesn't have Item weight are FDN52, FDK57, FDE52, FDQ60
     
### How we are going to fill that four missing values ?

In [10]:
key_names = ["Item_Fat_Content", "Item_Type", "Outlet_Size", "Outlet_Location_Type", "Outlet_Type"]
keys = ["Regular", "Dairy", "Medium", "Tier 3", "Supermarket Type3"]

temp = (df[(df[key_names] == keys).all(1)].drop_duplicates(subset ="Item_Identifier"))

temp[(temp['Item_MRP'] > 80.00) & (temp['Item_MRP'] < 100)]

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
4187,FDE52,NaN,Regular,0.029742,Dairy,88.9514,OUT027,1985,Medium,Tier 3,Supermarket Type3,3453.5046
5639,FDZ02,6.905,Regular,0.037963,Dairy,97.5726,OUT027,1985,Medium,Tier 3,Supermarket Type3,978.7260
5857,FDB04,11.350,Regular,0.062920,Dairy,88.9856,OUT027,1985,Medium,Tier 3,Supermarket Type3,3515.4240
7837,FDC16,11.500,Regular,0.020470,Dairy,88.2540,OUT027,1985,Medium,Tier 3,Supermarket Type3,2510.0660


In [11]:
temp[(temp['Item_MRP'] > 80.00) & (temp['Item_MRP'] < 100)].Item_Weight.mean()

9.918333333333333

### Assumption
If we identify all items that matches with following criteria
   1. Item_Fat_Content      : Regular
   2. Item_Type             : Dairy
   3. Outlet_Size           : Medium
   4. Outlet_Location_Type  : Tier 3
   5. Outlet_Type           : Supermarket Type3
   6. Item_MRP              : 80 < MRP < 100
   
From the above output cell we can say that,
   1. 6.5 < Item_Weight < 11.5

In [12]:
key_names = ["Item_Fat_Content", "Item_Type", "Outlet_Size", "Outlet_Location_Type", "Outlet_Type"]
keys = ["Regular", "Frozen Foods", "Medium", "Tier 3", "Supermarket Type3"]

temp = (df[(df[key_names] == keys).all(1)].drop_duplicates(subset ="Item_Identifier"))

temp[(temp['Item_MRP'] > 80.00) & (temp['Item_MRP'] < 100)]

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
927,FDN52,NaN,Regular,0.130933,Frozen Foods,86.9198,OUT027,1985,Medium,Tier 3,Supermarket Type3,1569.9564
7229,FDW16,17.35,Regular,0.041273,Frozen Foods,91.6804,OUT027,1985,Medium,Tier 3,Supermarket Type3,3858.9768


In [14]:
temp[(temp['Item_MRP'] > 80.00) & (temp['Item_MRP'] < 100)].Item_Weight.mean()

17.35

### Assumption
If we identify all items that matches with following criteria
   1. Item_Fat_Content      : Regular
   2. Item_Type             : Frozen Foods
   3. Outlet_Size           : Medium
   4. Outlet_Location_Type  : Tier 3
   5. Outlet_Type           : Supermarket Type3
   6. Item_MRP              : 80 < MRP < 100

From the above output cell we can say that,
   1. Item_Weight = 17.00 (apprx)

In [15]:
key_names = ["Item_Fat_Content", "Item_Type", "Outlet_Size", "Outlet_Location_Type", "Outlet_Type"]
keys = ["Low Fat", "Snack Foods", "Medium", "Tier 3", "Supermarket Type3"]

temp = (df[(df[key_names] == keys).all(1)].drop_duplicates(subset ="Item_Identifier"))

temp[(temp['Item_MRP'] > 110.00) & (temp['Item_MRP'] < 130.00)]

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
113,FDX34,6.195,Low Fat,0.071637,Snack Foods,121.7098,OUT027,1985,Medium,Tier 3,Supermarket Type3,4097.3332
279,FDZ10,17.850,Low Fat,0.044248,Snack Foods,126.2020,OUT027,1985,Medium,Tier 3,Supermarket Type3,3668.5580
598,FDL46,20.350,Low Fat,0.053795,Snack Foods,117.7466,OUT027,1985,Medium,Tier 3,Supermarket Type3,1414.1592
1922,FDK57,NaN,Low Fat,0.079904,Snack Foods,120.0440,OUT027,1985,Medium,Tier 3,Supermarket Type3,4434.2280
2563,FDV21,11.500,Low Fat,0.170254,Snack Foods,124.8704,OUT027,1985,Medium,Tier 3,Supermarket Type3,2127.8968
3306,FDA10,20.350,Low Fat,0.141129,Snack Foods,121.6072,OUT027,1985,Medium,Tier 3,Supermarket Type3,3430.2016
3503,FDL45,15.600,Low Fat,0.037505,Snack Foods,126.2704,OUT027,1985,Medium,Tier 3,Supermarket Type3,4255.7936
3661,FDF46,7.070,Low Fat,0.093218,Snack Foods,116.7834,OUT027,1985,Medium,Tier 3,Supermarket Type3,4952.8862
4078,FDE58,18.500,Low Fat,0.000000,Snack Foods,119.8124,OUT027,1985,Medium,Tier 3,Supermarket Type3,1659.1736
4537,FDN10,11.500,Low Fat,0.045900,Snack Foods,118.9124,OUT027,1985,Medium,Tier 3,Supermarket Type3,2844.2976


In [16]:
temp[(temp['Item_MRP'] > 110.00) & (temp['Item_MRP'] < 130.00)].Item_Weight.mean()

14.300000000000002

In [17]:
key_names = ["Item_Fat_Content", "Item_Type", "Outlet_Size", "Outlet_Location_Type", "Outlet_Type"]
keys = ["Regular", "Baking Goods", "Small", "Tier 1", "Grocery Store"]

temp = (df[(df[key_names] == keys).all(1)].drop_duplicates(subset ="Item_Identifier"))

temp[(temp['Item_MRP'] > 110.00) & (temp['Item_MRP'] < 130.00)]

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
3390,FDG12,6.635,Regular,0.000000,Baking Goods,121.3098,OUT019,1985,Small,Tier 1,Grocery Store,120.5098
5022,FDQ60,NaN,Regular,0.191501,Baking Goods,121.2098,OUT019,1985,Small,Tier 1,Grocery Store,120.5098


In [18]:
temp[(temp['Item_MRP'] > 110.00) & (temp['Item_MRP'] < 130.00)].Item_Weight.mean()

6.635

### Assumption
If we identify all items that matches with following criteria
   1. Item_Fat_Content      : Regular
   2. Item_Type             : Baking Goods
   3. Outlet_Size           : Small
   4. Outlet_Location_Type  : Tier 1
   5. Outlet_Type           : Grocery Store
   6. Item_MRP              : 110 < MRP < 130

From the above output cell we can say that,
   1. Item_Weight = 6.00 (apprx)

In [20]:
df.Item_Weight.fillna(value=df.Item_Weight.mean(), inplace=True)

In [21]:
df.isnull().sum()

Item_Identifier                 0
Item_Weight                     0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64